## Rename original files for easy parsing (Drop DLC part from the file names)

In [3]:
import os
import re

folder = r"C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Analyzed-videos-filtered-temp"

# Define known multi-word city names to fix
city_names = [
    'El_Paso', 'New_York', 'New_Jersey', 'New_Orleans',
    'Las_Cruces', 'Ft_Worth'
]
city_replacements = {name: name.replace('_', '') for name in city_names}

for file in os.listdir(folder):
    if not file.endswith(".csv"):
        continue

    original_path = os.path.join(folder, file)
    new_name = file

    # Step 1: Remove DLC suffix (if it still exists)
    new_name = re.sub(r'DLC_resnet50_.*?_filtered', '', new_name)

    # Step 2: Replace 'None_#' with 'None#'
    new_name = re.sub(r'None_(\d+)', r'None\1', new_name)

    # Step 3: Replace multi-word city names
    for old, new in city_replacements.items():
        new_name = new_name.replace(old, new)

    # Skip if no changes needed
    if new_name == file:
        continue

    new_path = os.path.join(folder, new_name)

    # Safety: skip if destination file already exists
    if os.path.exists(new_path):
        print(f"❗Skipped (target exists): {new_name}")
        continue

    # Rename file
    os.rename(original_path, new_path)
    print(f"✅ Renamed: {file}  -->  {new_name}")


✅ Renamed: Toy_Only_Gen_2_3_4_25_S1_Y_CNO__Austin_Houston_Toronto_Berlin_Trial_1_bottom_rightDLC_resnet50_DLCDec25shuffle1_1140000_filtered.csv  -->  Toy_Only_Gen_2_3_4_25_S1_Y_CNO__Austin_Houston_Toronto_Berlin_Trial_1_bottom_right.csv
✅ Renamed: Toy_Only_Gen_2_3_4_25_S1_Y_CNO__Austin_Houston_Toronto_Berlin_Trial_1_top_leftDLC_resnet50_DLCDec25shuffle1_1140000_filtered.csv  -->  Toy_Only_Gen_2_3_4_25_S1_Y_CNO__Austin_Houston_Toronto_Berlin_Trial_1_top_left.csv
✅ Renamed: Toy_Only_Gen_2_3_4_25_S1_Y_CNO__Austin_Houston_Toronto_Berlin_Trial_1_top_rightDLC_resnet50_DLCDec25shuffle1_1140000_filtered.csv  -->  Toy_Only_Gen_2_3_4_25_S1_Y_CNO__Austin_Houston_Toronto_Berlin_Trial_1_top_right.csv
✅ Renamed: Toy_Only_Gen_2_3_4_25_S2_Y_CNO__Denver_El_Paso_Waco_None_4_Trial_1_bottom_leftDLC_resnet50_DLCDec25shuffle1_1140000_filtered.csv  -->  Toy_Only_Gen_2_3_4_25_S2_Y_CNO__Denver_ElPaso_Waco_None4_Trial_1_bottom_left.csv
✅ Renamed: Toy_Only_Gen_2_3_4_25_S2_Y_CNO__Denver_El_Paso_Waco_None_4_Trial_

## Separate name block with '____'

In [4]:
import os
import re

folder = r"C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Analyzed-videos-filtered-temp"

for file in os.listdir(folder):
    if not file.endswith(".csv"):
        continue

    if "____Trial" in file:
        continue  # Already processed

    # Match the position of Trial_#
    trial_match = re.search(r'_Trial_\d+', file)
    if not trial_match:
        print(f"❌ Skipped (no Trial found): {file}")
        continue

    trial_start = trial_match.start()
    before_trial = file[:trial_start]
    after_trial = file[trial_start:]  # Includes _Trial_1...

    tokens = before_trial.split('_')
    if len(tokens) < 4:
        print(f"❌ Skipped (not enough tokens before Trial): {file}")
        continue

    # Get last 4 tokens = animal block
    animal_tokens = tokens[-4:]
    animal_block = '_'.join(animal_tokens)

    # Trim trailing underscore before animal block (if any)
    prefix = '_'.join(tokens[:-4])
    if prefix.endswith('_'):
        prefix = prefix[:-1]  # Remove trailing underscore

    # Rebuild filename
    new_file = f"{prefix}____{animal_block}____{after_trial}"

    original_path = os.path.join(folder, file)
    new_path = os.path.join(folder, new_file)

    if os.path.exists(new_path):
        print(f"⚠️ Skipped (target exists): {new_file}")
        continue

    os.rename(original_path, new_path)
    print(f"✅ Renamed:\n{file}  -->\n{new_file}\n")


✅ Renamed:
Toy_Only_Gen_2_3_4_25_S1_Y_CNO__Austin_Houston_Toronto_Berlin_Trial_1_bottom_right.csv  -->
Toy_Only_Gen_2_3_4_25_S1_Y_CNO____Austin_Houston_Toronto_Berlin_____Trial_1_bottom_right.csv

✅ Renamed:
Toy_Only_Gen_2_3_4_25_S1_Y_CNO__Austin_Houston_Toronto_Berlin_Trial_1_top_left.csv  -->
Toy_Only_Gen_2_3_4_25_S1_Y_CNO____Austin_Houston_Toronto_Berlin_____Trial_1_top_left.csv

✅ Renamed:
Toy_Only_Gen_2_3_4_25_S1_Y_CNO__Austin_Houston_Toronto_Berlin_Trial_1_top_right.csv  -->
Toy_Only_Gen_2_3_4_25_S1_Y_CNO____Austin_Houston_Toronto_Berlin_____Trial_1_top_right.csv

✅ Renamed:
Toy_Only_Gen_2_3_4_25_S2_Y_CNO__Denver_ElPaso_Waco_None4_Trial_1_bottom_left.csv  -->
Toy_Only_Gen_2_3_4_25_S2_Y_CNO____Denver_ElPaso_Waco_None4_____Trial_1_bottom_left.csv

✅ Renamed:
Toy_Only_Gen_2_3_4_25_S2_Y_CNO__Denver_ElPaso_Waco_None4_Trial_1_bottom_right.csv  -->
Toy_Only_Gen_2_3_4_25_S2_Y_CNO____Denver_ElPaso_Waco_None4_____Trial_1_bottom_right.csv

✅ Renamed:
Toy_Only_Gen_2_3_4_25_S2_Y_CNO__Denver_E

### Validation for extract_animal_name

In [20]:
import sys
sys.path.append('../Python_scripts')

from extract_animal_name import extract_animal_name

import os

# Folder path
folder = r"C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Analyzed-videos-filtered"
csv_files = [f for f in os.listdir(folder) if f.endswith(".csv")]

for file in csv_files[0:10]:
    # print(file)
    if file.endswith(".csv"):
        animal_name = extract_animal_name(file)
        if animal_name is not None:
            print(f"{file} --> {animal_name}")
        else:
            print(f"[SKIPPED] Pattern not matched: {file}")


Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_bottom_left.csv --> Toronto
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_bottom_right.csv --> Berlin
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_top_left.csv --> Houston
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_top_right.csv --> Austin
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_bottom_left.csv --> Waco
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_bottom_right.csv --> FtWorth
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_top_left.csv --> Denver
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_top_right.csv --> ElPaso
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S3___

### Validation for extract_maze_number

In [22]:
import sys
sys.path.append('../Python_scripts')

from extract_maze_number import extract_maze_number

import os

# Folder path
folder = r"C:\DeepLabCutProjects\DLC-Atanu-2024-12-25\Analyzed-videos-filtered"
csv_files = [f for f in os.listdir(folder) if f.endswith(".csv")]

for file in csv_files[0:10]:
    # print(file)
    if file.endswith(".csv"):
        maze_number = extract_maze_number(file)
        if maze_number is not None:
            print(f"{file} --> {maze_number}")
        else:
            print(f"[SKIPPED] Pattern not matched: {file}")

Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_bottom_left.csv --> 3
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_bottom_right.csv --> 4
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_top_left.csv --> 1
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S1____Houston_Austin_Toronto_Berlin_____Trial_1_top_right.csv --> 2
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_bottom_left.csv --> 3
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_bottom_right.csv --> 4
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_top_left.csv --> 1
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S2____Denver_ElPaso_Waco_FtWorth_____Trial_1_top_right.csv --> 2
Cre_and_WT_Habituation_Food_and_Maze_12_26_24_S3____Orlando_Tampa_Dallas_None4_____Trial_1_b